In [1]:
import pandas as pd
import numpy as np

In [2]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


### Признаки в данных:

Pregnancies — количество беременностей.

Glucose — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе.

BloodPressure — диастолическое артериальное давление (мм рт. ст.).

SkinThickness — толщина кожной складки трицепса (мм).

Insulin — двухчасовой сывороточный инсулин (ме Ед/мл).

BMI — индекс массы тела

DiabetesPedigreeFunction — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).

Age — возраст.

Outcome — наличие диабета (0 — нет, 1 — да).

In [4]:
diabetes.info()
diabetes.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               778 non-null    int64  
 1   Glucose                   778 non-null    int64  
 2   BloodPressure             778 non-null    int64  
 3   SkinThickness             778 non-null    int64  
 4   Insulin                   778 non-null    int64  
 5   BMI                       778 non-null    float64
 6   DiabetesPedigreeFunction  778 non-null    float64
 7   Age                       778 non-null    int64  
 8   Outcome                   778 non-null    int64  
 9   Gender                    778 non-null    object 
dtypes: float64(2), int64(7), object(1)
memory usage: 60.9+ KB


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000
mean,3.848329,120.822622,69.035990,20.457584,79.521851,31.982262,0.470871,33.317481,0.344473
std,3.360782,31.883264,19.432323,15.954452,114.862405,7.853917,0.330669,11.816300,0.475502
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.241500,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,27.000000,32.000000,0.370000,29.000000,0.000000
75%,6.000000,140.000000,80.000000,32.000000,126.750000,36.500000,0.625500,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
dupl_columns = list(diabetes.columns)

mask = diabetes.duplicated(subset=dupl_columns)
diabetes_duplicates = diabetes[mask]
print(f'Число найденных дубликатов: {diabetes_duplicates.shape[0]}')

diabetes_dedupped = diabetes.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {diabetes_dedupped.shape[0]}')

Число найденных дубликатов: 10
Результирующее число записей: 768


In [7]:
def find_non_information_cols(data, threshold=0.95):

    #список неинформативных признаков
    low_information_cols = [] 

    #цикл по всем столбцам
    for col in data.columns:
        #наибольшая относительная частота в признаке
        top_freq = data[col].value_counts(normalize=True).max()
        #доля уникальных значений от размера признака
        nunique_ratio = data[col].nunique() / data[col].count()
        # сравниваем наибольшую частоту с порогом
        if top_freq > threshold:
            low_information_cols.append(col)
            print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
        # сравниваем долю уникальных значений с порогом
        if nunique_ratio > threshold:
            low_information_cols.append(col)
            print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
            
    return low_information_cols

low_information_cols = find_non_information_cols(diabetes_dedupped)

cleaned_diabetes = diabetes_dedupped.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {cleaned_diabetes.shape[1]}')

Gender: 100.0% одинаковых значений
Результирующее число признаков: 9


In [17]:
def nan_replace(arg):
    if arg == 0:
        return np.nan
    else:
        return arg
    
cleaned_diabetes['Insulin'] = cleaned_diabetes['Insulin'].apply(nan_replace)
cleaned_diabetes['Glucose'] = cleaned_diabetes['Glucose'].apply(nan_replace)
cleaned_diabetes['BloodPressure'] = cleaned_diabetes['BloodPressure'].apply(nan_replace)
cleaned_diabetes['SkinThickness'] = cleaned_diabetes['SkinThickness'].apply(nan_replace)
cleaned_diabetes['BMI'] = cleaned_diabetes['BMI'].apply(nan_replace)

# получаем столбцы с пропусками и процент пропуска в тих столбцах

cols_null_percent = cleaned_diabetes.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)


Insulin          48.697917
SkinThickness    29.557292
BloodPressure     4.557292
BMI               1.432292
Glucose           0.651042
dtype: float64

In [18]:
# функция удаляет столбцы(признаки) где число пропусков более порогового значение (по умлочанию 0.7 или 30%)
def del_nan_cols(data, thresh=0.7):
    #отбрасываем столбцы с числом пропусков более 30% (100-70) по умолчанию
    n = data.shape[0] #число строк в таблице
    thresh = n*thresh
    data = data.dropna(thresh=thresh, axis=1)
    
    return data

In [19]:
cleaned_diabetes = del_nan_cols(cleaned_diabetes) # удаляем столбцы где пропусков более 30%
cleaned_diabetes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 768 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   763 non-null    float64
 2   BloodPressure             733 non-null    float64
 3   SkinThickness             541 non-null    float64
 4   BMI                       757 non-null    float64
 5   DiabetesPedigreeFunction  768 non-null    float64
 6   Age                       768 non-null    int64  
 7   Outcome                   768 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 54.0 KB


In [20]:
#отбрасываем строки с числом пропусков более 2 в строке
m = cleaned_diabetes.shape[1] #число признаков
cleaned_diabetes = cleaned_diabetes.dropna(thresh=m-2, axis=0)
cleaned_diabetes.shape[0]

761

In [23]:
#создаём словарь 'имя_столбца': число (признак), на который надо заменить пропуски 
values = {
    'Glucose': cleaned_diabetes['Glucose'].median(),
    'BloodPressure': cleaned_diabetes['BloodPressure'].median(),
    'SkinThickness': cleaned_diabetes['SkinThickness'].median(),
    'BMI': cleaned_diabetes['BMI'].median(),
}

cleaned_diabetes = cleaned_diabetes.fillna(values)

cleaned_diabetes['SkinThickness'].mean()

29.109067017082786

In [37]:
def outliers_iqr_mod(data, feature, log_scale=False, left=1.5, right=1.5, zero_expect=False):
    
    if log_scale:
        
        x = np.log(data[feature]+int(zero_expect))
    else:
        x = data[feature]
 
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [25]:
outliers, cleaned = outliers_iqr_mod(cleaned_diabetes, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


In [38]:
def outliers_z_score_mod(data, feature, log_scale=False, left=3, right=3, zero_expect=False):
    if log_scale:
        x = np.log(data[feature]+int(zero_expect))
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [42]:
outliers, cleaned = outliers_z_score_mod(cleaned_diabetes, 'SkinThickness')
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

outliers, cleaned = outliers_iqr_mod(cleaned_diabetes, 'DiabetesPedigreeFunction')
print(f'Число выбросов в DiabetesPedigreeFunction: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

outliers, cleaned = outliers_iqr_mod(cleaned_diabetes, 'DiabetesPedigreeFunction', log_scale=True, zero_expect=False)
print(f'Число выбросов в DiabetesPedigreeFunction в log-масштабе: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757
Число выбросов в DiabetesPedigreeFunction: 29
Результирующее число записей: 732
Число выбросов в DiabetesPedigreeFunction в log-масштабе: 0
Результирующее число записей: 761
